In [3]:
#Libraries

import numpy as np
import cv2
from scipy.spatial import distance
import statistics

In [4]:
#---------------------------------------------------------------------
#Display the image
def display_image(name,image):
    
    cv2.imshow(name,image)
    cv2.waitKey(0)
#---------------------------------------------------------------------

In [17]:
def image_processing(image):
    
    #Original Image
    img = image
    img_copy = img.copy()
    
    #Grayscaling the image
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    display_image("Grayscale",gray)
    
    #Thresholding the image to convert all the pixels with intensity above 90 (gray) to 22 (white). Why 90? Its the ideal value for this particular image
    _,thresh = cv2.threshold(gray,90,255,cv2.THRESH_BINARY)
    display_image("Theshold",thresh)
    
    #Median blur to remove the noise
    median = cv2.medianBlur(thresh,5)
    display_image("Blur",median)
    
    #Canny edge detector to detect the edges
    canny = cv2.Canny(median, 200, 100)
    
    #Detecting circles using Hough circles. MaxRadius is not given yet since the distance is not finalized
    circles = cv2.HoughCircles(canny, cv2.HOUGH_GRADIENT, 1, 20, param1 = 200, param2 = 20, minRadius = 10, maxRadius = 0)
    
    if circles is not None:
        
        #Obtaining the hough circle (The biggest circle is always the first output)
        inner_circle = np.uint16(np.around(circles[0][0])).tolist()
        
        #Drawing a circle over the hough circle outputs
        cv2.circle(image, (inner_circle[0], inner_circle[1]), inner_circle[2], (0, 255, 0), 1)
        display_image("Hough",image)
        
        # Removing all the artifacts using masking(removing the black noise by cropping out the circle and pasting it onto a white image)
        white_image = np.full((img.shape[0], img.shape[1]), 255, dtype=np.uint8)
        cv2.circle(white_image, (inner_circle[0], inner_circle[1]), inner_circle[2]+8, (0, 0, 0), -1)
        display_image('Mask',white_image)
        
        #Printing the obtain iris region for reference
        roi = cv2.bitwise_or(median,white_image)
        display_image('roi',roi)
        
        #Applying edge detection over the region of interest
        canny_image = cv2.Canny(roi, 200, 100)
        display_image("canny_roi",canny_image)
        
        #Detecting the contours and drawing them on the image to see the region obtained
        contours, hierarchy = cv2.findContours(canny_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        detected_contour = cv2.drawContours(image.copy(), contours, -1, (0,255,0), 1)
        display_image("Contours",detected_contour)
    
        #Drawing a point in the center of the detected iris for reference
        x = cv2.circle(image.copy(),(inner_circle[0], inner_circle[1]), 1, (255, 100, 0), 2)
        display_image("Center",x)
        
        
        #Finding all the non zero values(white pixels) from the canny image to measure the distance between the 
        pixelpoints = cv2.findNonZero(canny_image)
        #print(pixelpoints)
        
        #List to store all the distance values in pixels
        distance_list = []  
        
        for i,point in enumerate(pixelpoints):
            
            contour_point = (point[0][0],point[0][1])
            center = (inner_circle[0],inner_circle[1])
            dist = distance.euclidean(contour_point,center)
            distance_list.append(dist)
        #print(distance_list)
        
        #A list to store all the converted radius values
        radius_list = []
        for i,val in enumerate(distance_list):
            
            #Since the picture is taken from an ideal distance, therefore the radius is more or less equal to the original value
            radius = val * 0.2645833333 # 1 pixel = .2645833333 mm
            if radius > 7:
                continue
            else:
                #Adding the values which are below the 
                radius_list.append(radius)
        #print(radius_list)
        
        #The radius is obtained by finding the median of the list of selected values
        final_value = statistics.median(radius_list)
        print('Radius is : ',final_value)
        
        print("Diameter is ",final_value * 2)
        

In [18]:
image = cv2.imread('eye.jpg',1)
display_image("Original",image)
pp = image_processing(image)

Radius is :  6.776639817096015
Diameter is  13.55327963419203
